In [ ]:
import win32com.client # pip install pwin32
import os
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [ ]:
mes_actual_str = 'Julio'
mes_actual_int = '07'
hoy= datetime.now().day
mes_anterior_str = 'Junio'
mes_anterior_int = '06'

In [8]:
if len(str(hoy)) == 1:
    hoy = '0' + str(hoy)

In [173]:
ruta_suspension = 'C:/Users/BBARTOLOMEE/Desktop/SMS/WA PUSH Suspension Lineas/2025'

In [174]:
def carpeta_ultima(ruta_suspension, mes_int, mes_str):
    band = False
    for archivo in os.listdir(ruta_suspension):
        if archivo.lower().startswith(mes_actual_int):
            band = True
    if band is False:
        print('Archivo del mes actual no encontrado, se crea uno')
        os.mkdir(ruta_suspension + '/' + mes_int + '. ' +mes_str)
    else:
        print('Existe carpeta del mes actual')
    
    

In [175]:
carpeta_ultima(ruta_suspension, mes_actual_int, mes_actual_str)

Existe carpeta del mes actual


In [176]:
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
inbox = outlook.GetDefaultFolder(6)
messages = inbox.Items
messages.Sort("[ReceivedTime]", True)

remitente= "pe-soporte@ep.net.pe"
palabra_clave_asunto = "Suspensión"

download_folder = os.path.join(os.getcwd(), "adjuntos_filtrados")
os.makedirs(download_folder, exist_ok=True)


<h4>Buscamos el correo y guardamos el excel adjunto</h4>

In [177]:
for message in messages:
    try:
        if (remitente.lower() in message.SenderEmailAddress.lower() and 
            palabra_clave_asunto.lower() in message.Subject.lower()):
            
            print(f"Correo encontrado: {message.Subject} - {message.SenderEmailAddress}")
            
            if message.Attachments.Count > 0:
                for i in range(1, message.Attachments.Count + 1):
                    attachment = message.Attachments.Item(i)
                    if attachment.FileName.endswith((".xlsx", ".xls")):
                        filepath = os.path.join(download_folder, attachment.FileName)
                        attachment.SaveAsFile(filepath)
                        print(f"Archivo guardado: {filepath}")

                        # Leer Excel
                        df = pd.read_excel(filepath, sheet_name = 'FILTRO DUPLICIDADES')
                        print(df.head())
                        break
            break
    except Exception as e:
        print("Error al procesar un mensaje:", e)
else:
    print("No se encontró un correo que cumpla los criterios.")

In [178]:
Solicitante = ''
if 'movistar' in message.Subject.lower():
     Solicitante = 'Movistar'
elif 'claro' in message.Subject.lower():
     Solicitante = 'Claro'
else:
     pass

In [179]:
cuerpo = message.Body
patron = r"suspensión.*?a más tardar el\s+(.*?)\s+a las\s+(\d{1,2}:\d{2})\s*hrs"
coincidencia = re.search(patron, cuerpo, re.IGNORECASE | re.DOTALL)

if coincidencia:
    dia_y_fecha = coincidencia.group(1).strip()    
    print(dia_y_fecha)
else:
    print("No se encontró la información de suspensión en el cuerpo del correo.")


lunes 7 de julio


In [180]:
lista = dia_y_fecha.split(' ')
dia = int(lista[1])
mes = ''
anio = 2025
if lista[3] == mes_actual_str.lower():
    mes = mes_actual_int
else:
    mes = mes_anterior_int

fecha_dt = datetime(anio, int(mes), dia).strftime("%d/%m/%Y")


In [ ]:
df

In [182]:
df_rucs = df[df['SUB SEGMENTO']=='RUC 20']
df_rucs.reset_index(drop=True,inplace=True)

In [183]:
df_rucs = df_rucs[['TELEFONO (51)', 'CARGO FIJO', 'ESTADO', 'SUB SEGMENTO']]
df_rucs['Solicitante'] = Solicitante
df_rucs['Medio Comunicación'] = 'SMS'
df_rucs['Fecha Suspensión'] = fecha_dt
df_rucs.rename(columns={'TELEFONO (51)':'TELEFONO','CARGO FIJO':'DEUDA','SUB SEGMENTO':'Tipo RUC'},inplace=True)

In [ ]:
df_rucs

In [185]:
#Falta cruzar con suscriptores, afiliaciones, subir plantilla y copiar columnas

In [10]:
ruta_carpeta_suscriptores = 'C:/Users/BBARTOLOMEE/OneDrive - Entel Peru S.A/Afiliaciones/2025/'

In [11]:
def carpeta_mes_suscriptores(ruta_suspension):
    band = False
    for archivo in os.listdir(ruta_suspension):
        if archivo.lower().startswith(mes_actual_int):
            band = True
    return band

In [12]:
carpeta = carpeta_mes_suscriptores(ruta_carpeta_suscriptores)

if carpeta:
    ruta_suscriptores = f'{ruta_carpeta_suscriptores}{mes_actual_int}. {mes_actual_str}/Suscriptores'
else:
    ruta_suscriptores = f'{ruta_carpeta_suscriptores}{mes_anterior_int}. {mes_anterior_str}/Suscriptores'   

In [189]:
def ultimo_excel_suscriptores(ruta_carpeta):
    archivos_validos = []

    for archivo in os.listdir(ruta_carpeta):
        if archivo.lower().endswith('.xlsx') and "Suscriptores." in archivo:
            try:
                partes = archivo.split(".")
                if len(partes) >= 3:
                    dia = int(partes[1])
                    mes = int(partes[2])
                    archivos_validos.append(((mes, dia), archivo))
            except ValueError:
                print(f"'{archivo}' tiene un formato de fecha inválido.")

    if not archivos_validos:
        print("No se encontraron archivos con fechas válidas.")
        return None

    archivos_validos.sort(reverse=True)
    return archivos_validos[0][1]

In [190]:
ultimo_suscriptores = ultimo_excel_suscriptores(ruta_suscriptores)
print(ultimo_suscriptores)

Suscriptores.01.07.xlsx


In [191]:
ruta_completa_suscriptores = ruta_suscriptores + '/' + ultimo_suscriptores
df_suscriptores = pd.read_excel(ruta_completa_suscriptores)

df_suscriptores.drop_duplicates(subset = 'TELEFONO', inplace = True)
df_cruce_suscriptores = pd.merge(df_rucs, df_suscriptores[['TELEFONO','FECHAACTIVACIONCONTRATO','DOCUMENTO']], on = 'TELEFONO', how = 'left')

In [192]:
#df_rucs.info()
#print('\t-----------------------------\n'*6)
#df_cruce_suscriptores.info()

In [ ]:
df_cruce_suscriptores.head(7)

In [194]:
ruta_carpeta_afiliaciones = 'C:/Users/BBARTOLOMEE/OneDrive - Entel Peru S.A/Afiliaciones/2025'

In [ ]:
carpeta_afiliaciones = carpeta_mes_suscriptores(ruta_carpeta_afiliaciones)

if carpeta_afiliaciones:
    ruta_afiliaciones = f'{ruta_carpeta_afiliaciones}{mes_actual_int}. {mes_actual_str}/'
    nombre_hoja = 'Afiliaciones '+ mes_actual_str + ' 25'
else:
    ruta_afiliaciones = f'{ruta_carpeta_afiliaciones}{mes_anterior_int}. {mes_anterior_str}/'   
    nombre_hoja = 'Afiliaciones '+ mes_anterior_str + ' 25'

In [196]:
#Buscar archivo afiliaciones para obtener su nombre actual
def buscar_nombre_afiliaciones(ruta_afiliaciones):
        for archivo in os.listdir(ruta_afiliaciones):
                if archivo.lower().startswith('afiliaciones') and archivo.lower().endswith('25.xlsx'):
                        nombre = archivo
                        return nombre

In [197]:
nombre_afiliaciones = buscar_nombre_afiliaciones(ruta_afiliaciones)

ruta_completa_afiliaciones = ruta_afiliaciones + nombre_afiliaciones 

In [198]:
df_afiliaciones = pd.read_excel(ruta_completa_afiliaciones, sheet_name= nombre_hoja)

In [199]:
df_afiliaciones = df_afiliaciones.rename(columns={'RUCCOMPANIA': 'DOCUMENTO'})
df_cruce_afiliaciones = pd.merge(df_cruce_suscriptores, df_afiliaciones[['DOCUMENTO','SUG_SEGM']], on = 'DOCUMENTO', how = 'left')

In [ ]:
df_cruce_afiliaciones.head(5)

In [201]:
df_cruce_afiliaciones['Fecha activacion'] = df_cruce_afiliaciones['FECHAACTIVACIONCONTRATO'].dt.strftime('%d/%m/%Y')
df_final = df_cruce_afiliaciones[['TELEFONO','Fecha activacion', 'DOCUMENTO', 'Tipo RUC', 'ESTADO', 'SUG_SEGM', 'DEUDA', 'Solicitante', 'Fecha Suspensión', 'Medio Comunicación']].copy()

In [ ]:
df_final

In [203]:
df_base_suspension = pd.read_excel('Plantilla.xlsx')

df_base_suspension['Celular'] = df_final['TELEFONO']
df_base_suspension['Operador1'] = df_final['Solicitante']
df_base_suspension['Número'] = df_final['TELEFONO']
df_base_suspension['Fecha corte'] = df_final['Fecha Suspensión']
df_base_suspension['Monto'] = df_final['DEUDA']
df_base_suspension['Operador2'] = df_final['Solicitante']

condicion = df_base_suspension['Monto'].isna()
df_base_suspension.loc[condicion, 'Monto'] = 'pago'

C:\Users\BBARTOLOMEE\AppData\Local\Temp\ipykernel_81696\268673776.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'pago' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_base_suspension.loc[condicion, 'Monto'] = 'pago'


In [204]:
df_final.fillna("#N/D", inplace=True)

C:\Users\BBARTOLOMEE\AppData\Local\Temp\ipykernel_81696\42174674.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '#N/D' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_final.fillna("#N/D", inplace=True)


In [ ]:
df_base_suspension

In [ ]:
df_base_suspension.to_excel(f'C:/Users/BBARTOLOMEE/Desktop/SMS/WA PUSH Suspension Lineas/2025/{mes_actual_int}. {mes_actual_str}/Suspension Lineas {hoy}.{mes_actual_int}.xlsx', index = False)
df_final.to_excel('Añadir a base general.xlsx', index = False)